In [18]:
import pandas as pd
import datasource.gejala as gj
import datasource.penyakit as pk

df: pd.DataFrame = pd.read_csv(
  filepath_or_buffer="penyakit-ayam.csv"
)

# df.index = df.index + 1

COL_KASUS: str = "Kasus"
COL_G1: str = "G1"
COL_G2: str = "G2"
COL_G3: str = "G3"
COL_G4: str = "G4"
COL_G5: str = "G5"
COL_G6: str = "G6"
COL_G7: str = "G7"
COL_G8: str = "G8"
COL_PENYAKIT: str = "Penyakit"

# row2: pd.Series = df.iloc[1]

# print(row2[COL_G2])

# for idx, row in df.iterrows():
#   print(f"Index: {idx}")
#   print(row)  # Access each row's data as a Series
#   print("------") 

# g1 = df[df[COL_G1] == 1]

# print(g1)

# result = filtered_df['Penyakit'].value_counts()

penyakit = df[COL_PENYAKIT]
# for i, v in penyakit.items():
#   print(i," : ",v)

for k, v in gj.GEJALA.items():
  gejala = df[k]
  totalGejala: int = (df[k] == 1).sum()
  print("Gelaja : "+v)
  print("Total :", totalGejala)
  print(gejala)
  print("-------------")


Gelaja : Gangguan Syaraf
Total : 6
0     1
1     0
2     1
3     0
4     1
5     1
6     0
7     1
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    1
19    0
Name: G1, dtype: int64
-------------
Gelaja : Diare
Total : 15
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     0
10    1
11    1
12    1
13    1
14    0
15    0
16    0
17    1
18    0
19    1
Name: G2, dtype: int64
-------------
Gelaja : Nafsu Makan Menurun
Total : 12
0     0
1     1
2     0
3     1
4     0
5     0
6     1
7     1
8     0
9     1
10    1
11    0
12    1
13    1
14    1
15    1
16    1
17    1
18    0
19    0
Name: G3, dtype: int64
-------------
Gelaja : Merejan
Total : 7
0     1
1     1
2     1
3     0
4     0
5     0
6     0
7     0
8     1
9     0
10    0
11    1
12    0
13    0
14    1
15    1
16    0
17    0
18    0
19    0
Name: G4, dtype: int64
-------------
Gelaja : Tubuh Gemetar
Total : 6
0     1
1     0
2     1
3     0
4     0
5     1
6  